In [1]:
%pip install ipykernel
%pip install yfinance
%pip install pymoo
%pip install pandas numpy matplotlib seaborn
%pip install scikit-learn
%pip install tensorflow
%pip install keras
%pip install talib-binary
%pip install TA-Lib

ERROR: Could not find a version that satisfies the requirement talib-binary (from versions: none)
ERROR: No matching distribution found for talib-binary


KeyboardInterrupt: 

In [ ]:
import warnings
import yfinance as yf
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import Huber

from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

# --- Setup ---
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

In [ ]:
import yfinance as yf
import pandas as pd

# Define the tickers for the selected assets, including SPY
# NOTE: SPY is included in the download for BENCHMARK comparison only
#       It will be EXCLUDED from portfolio optimization and Bi-LSTM training
#       SPY is only used to compare performance against the optimized portfolio

#tickers = ['QQQ', 'SCHD', 'GLD', 'USDU', 'IEF', 'VNQ', 'SPY'] # Replaced BTC-USD with FXE (Invesco CurrencyShares Euro Trust)
#tickers = ['QQQ', 'XLE', 'XLP', 'VEA', 'GLD', 'FXE', 'SCHH', 'SCHD', 'IEF', 'SPY']
# Porfolio selection based on applying diverisification principles
tickers = ['AAPL', 'GOOG', 'MSFT', 'NVDA',
           'AMZN', 'AMD', 'INTC', 'META',
           'COST', 'PG', 'KO', 'PEP', 'WMT',
           'CME', 'AVGO', 'PFE', 'ABBV', 'HD',
           'XOM', 'VDE',
           'SCHD', 'VYM',
           'VWO', 'VEA',
           'GLD', 'SLV',
           'FXY', 'FDIVX',
           'TLT', 'SPLB',
           'SPY']  # <-- SPY is BENCHMARK ONLY (excluded from portfolio)
# Download historical data for the last 25 years
data = yf.download(tickers, period='25y') # Changed period to 10y

# We'll focus on the 'Close' price since 'Adj Close' is not directly available
# Use dropna(how='all') to only remove rows where ALL tickers are NaN
# This preserves SPY and other tickers even if some have missing data
close_data = data['Close'].dropna(how='all')

# Forward fill any remaining NaN values to handle missing data points
close_data = close_data.fillna(method='ffill').fillna(method='bfill')

# Ensure close_data index is timezone-naive for consistent alignment
if close_data.index.tz is not None:
    close_data.index = close_data.index.tz_localize(None)

display(close_data.tail())
print(f"\n✓ Close data shape: {close_data.shape}")
print(f"✓ Tickers included: {list(close_data.columns)}")
print(f"✓ SPY included: {'SPY' in close_data.columns}")

In [ ]:
# Calculate daily returns
daily_returns = close_data.pct_change().dropna()

# Calculate cumulative returns
cumulative_returns = (1 + daily_returns).cumprod() - 1

display(cumulative_returns.tail())
print(f"\n✓ Cumulative returns shape: {cumulative_returns.shape}")
print(f"✓ SPY in cumulative_returns: {'SPY' in cumulative_returns.columns}")

In [ ]:
import matplotlib.pyplot as plt

# Plot cumulative returns
plt.figure(figsize=(12, 8))
for ticker in cumulative_returns.columns:
    plt.plot(cumulative_returns.index, cumulative_returns[ticker], label=ticker)

plt.title('Cumulative Returns of Selected Assets and SPY (Last 10 Years)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import yfinance as yf
import pandas as pd

# Exclude SPY from the list of portfolio tickers
portfolio_tickers = list(filter(lambda item: item != 'SPY', tickers))

# Initialize dividend_data with the full date range of close_data and all portfolio tickers
# Fill with 0.0 initially, as dividends are sparse.
dividend_data = pd.DataFrame(0.0, index=close_data.index, columns=portfolio_tickers)

for ticker in portfolio_tickers:
    try:
        dividends_series = yf.Ticker(ticker).dividends
        if not dividends_series.empty:
            # Ensure dividend index is timezone-naive for consistent alignment
            if dividends_series.index.tz is not None:
                dividends_series.index = dividends_series.index.tz_localize(None)

            # Reindex the dividend series to the full close_data index and fill NaNs with 0
            # Then assign it to the corresponding column in the pre-initialized dividend_data DataFrame
            dividend_data[ticker] = dividends_series.reindex(close_data.index, fill_value=0.0)

    except Exception as e:
        print(f"Could not retrieve dividend data for {ticker}: {e}")

# The dividend_data DataFrame is already filled with 0.0 for non-dividend days during reindexing,
# so a final fillna(0) might be redundant but doesn't hurt.
dividend_data = dividend_data.fillna(0)
display(dividend_data.tail())

In [ ]:
# To calculate the total dividends paid by an equal-weighted portfolio,
# we can sum the dividends paid by each asset on each date and divide by the number of assets.
# However, dividend payments are not daily. We'll resample to a more appropriate frequency, like monthly or quarterly,
# and calculate the sum of dividends paid within each period. Let's use a monthly frequency.
monthly_dividends = dividend_data.resample('M').sum()

annualized_dividends = monthly_dividends.resample('Y').sum()
display("Annualized Dividends Paid by Equal Weighted Portfolio (Last 10 Years):")
display(annualized_dividends)

annualized_yield = annualized_dividends / close_data[annualized_dividends.columns]

# Assuming an equal-weighted portfolio, the total dividend received by the portfolio in each month
# is the sum of dividends from each asset divided by the number of assets.
# We need to align this with the dates in our price data.
# For simplicity here, we'll calculate the cumulative sum of dividends based on the dividend payment dates.
# A more precise calculation would involve tracking the portfolio holdings over time.

# Let's calculate the cumulative sum of dividends over time
cumulative_dividends = monthly_dividends.sum(axis=1).cumsum()

display("Cumulative Dividends Paid by Equal Weighted Portfolio (Monthly):")
display(cumulative_dividends.tail())

In [ ]:
import pandas as pd

# Ensure dividend_data and close_data are available
if 'dividend_data' not in globals() or 'close_data' not in globals():
    raise ValueError("dividend_data or close_data not found. Please ensure these are loaded.")

# Calculate annual dividends paid per ticker
# We'll reindex dividend_data to close_data's index and then resample annually
aligned_daily_dividends = dividend_data.reindex(close_data.index).fillna(0)
annual_dividends_sum = aligned_daily_dividends.resample('YE').sum()

# Get year-end closing prices per ticker
# Find the last available close price for each year
annual_closing_prices = close_data.resample('YE').last()

# Calculate the annualized dividend yield for each year and ticker
# Handle potential division by zero by replacing 0s with NaN before division
annual_dividend_yield_per_ticker = annual_dividends_sum.div(annual_closing_prices.replace(0, pd.NA))

display("Annualized Dividend Yield (Annual Dividends / Year-End Price) per Ticker:")
display(annual_dividend_yield_per_ticker.fillna(0).tail())

# Optional: Display mean annual yield over the period for comparison
#display("\nAverage Annualized Dividend Yield (Mean over years) per Ticker:")
#display(annual_dividend_yield_per_ticker.mean().fillna(0).sort_values(ascending=False))

In [ ]:
import pandas as pd
import numpy as np

# Ensure required dataframes are available
if 'annual_dividend_yield_per_ticker' not in globals():
    raise ValueError("annual_dividend_yield_per_ticker is not defined. Please run the previous cell to calculate it.")
if 'annual_closing_prices' not in globals():
    raise ValueError("annual_closing_prices is not defined. Please run the previous cell to calculate it.")
if 'close_data' not in globals():
    raise ValueError("close_data is not defined. Please ensure historical price data is downloaded.")

# 1. Calculate the total annual dividends in dollar amounts
# We need to ensure indices and columns align for multiplication
# Reindex annual_closing_prices to match annual_dividend_yield_per_ticker index and columns
aligned_annual_closing_prices = annual_closing_prices.reindex_like(annual_dividend_yield_per_ticker)
annual_dividend_dollars_per_ticker = annual_dividend_yield_per_ticker * aligned_annual_closing_prices

# Fill any NaNs that might result from missing data points in either source with 0
annual_dividend_dollars_per_ticker = annual_dividend_dollars_per_ticker.fillna(0)

# 2. Determine the number of trading days for each year
# Group close_data by year and count the number of entries
trading_days_per_year = close_data.groupby(close_data.index.year).size()

# Create a DataFrame to store the new aligned daily dividend data
aligned_daily_dividend_data_new = pd.DataFrame(0.0, index=close_data.index, columns=portfolio_tickers)

# 3. Distribute annual dividends evenly across trading days in each year
for year in annual_dividend_dollars_per_ticker.index.year.unique():
    if year in trading_days_per_year.index:
        num_days = trading_days_per_year[year]
        if num_days > 0:
            # Get the annual dividend dollars for the current year
            annual_dividends_for_year = annual_dividend_dollars_per_ticker.loc[annual_dividend_dollars_per_ticker.index.year == year].iloc[0]

            # Calculate the daily average dividend amount for this year
            daily_avg_dividends_for_year = annual_dividends_for_year / num_days

            # Assign this daily average to all trading days within that year
            year_mask = (aligned_daily_dividend_data_new.index.year == year)
            # Ensure alignment of columns and use .loc for assignment
            aligned_daily_dividend_data_new.loc[year_mask, daily_avg_dividends_for_year.index] = daily_avg_dividends_for_year.values

# Display the head and tail of the newly created DataFrame
display("New Aligned Daily Dividend Data (derived from annual yield/year-end price distributed daily):")
display(aligned_daily_dividend_data_new.tail())

# Update the global aligned_daily_dividend_data variable with the new calculation
aligned_daily_dividend_data = aligned_daily_dividend_data_new

# Update nsga_ii_data with the average annual dividend yield (re-calculating using this new data)
# Recalculate average_dividend_yield from the newly created aligned_daily_dividend_data
cols = list(portfolio_tickers)

# Reindex aligned_daily_dividend_data to close_data's index if not already (should be the case)
div_df_aligned = aligned_daily_dividend_data.reindex(close_data.index).fillna(0)

# Calculate annual dividends sum
annual_div_sum = div_df_aligned.resample('YE').sum()

# Get year-end closing prices per ticker for average yield calculation
annual_close_prices_for_yield = close_data.resample('YE').last()

# Calculate average annual yield based on new data
# Handle potential division by zero
avg_annual_yield = (annual_div_sum.div(annual_close_prices_for_yield.replace(0, np.nan))).mean(axis=0).reindex(cols).fillna(0)


In [ ]:
import matplotlib.pyplot as plt

# Plot cumulative dividends paid by the portfolio
plt.figure(figsize=(12, 8))
plt.plot(cumulative_dividends.index, cumulative_dividends)

plt.title('Cumulative Dividends Paid by Equal Weighted Portfolio (Last 10 Years)')
plt.xlabel('Date')
plt.ylabel('Cumulative Dividends')
plt.grid(True)
plt.show()

In [ ]:
# Exclude SPY from the portfolio
portfolio_assets = close_data.drop(columns=['SPY'])

# Calculate daily returns for the portfolio assets
portfolio_daily_returns = portfolio_assets.pct_change().dropna()

# For a simple equal-weighted portfolio, the portfolio daily return is the average of the asset daily returns
portfolio_returns = portfolio_daily_returns.mean(axis=1)

# Calculate cumulative returns of the portfolio
portfolio_cumulative_returns = (1 + portfolio_returns).cumprod() - 1

display("Portfolio Cumulative Returns (Equal Weighted):")
display(portfolio_cumulative_returns.tail())

In [ ]:
import numpy as np

# Calculate annualized volatility (assuming 252 trading days in a year)
annualized_volatility = portfolio_returns.std() * np.sqrt(252)

# Calculate Sharpe Ratio (assuming risk-free rate is 0 for simplicity in this example)
sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252)

display(f"Portfolio Annualized Volatility (Equal Weighted): {annualized_volatility:.4f}")
display(f"Portfolio Sharpe Ratio (Equal Weighted): {sharpe_ratio:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Plot cumulative returns with log scale
plt.figure(figsize=(12, 8))
for ticker in cumulative_returns.columns:
    plt.plot(cumulative_returns.index, cumulative_returns[ticker] + 1, label=ticker)  # +1 to avoid log(0)

plt.yscale('log')  # Enable log scale
plt.title('Cumulative Returns of Selected Assets and SPY (Log Scale)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return (1 + return, log scale)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, which='both', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# Calculate daily returns for SPY
spy_returns = daily_returns['SPY']

# Calculate Annualized Volatility for SPY (assuming 252 trading days in a year)
spy_annualized_volatility = spy_returns.std() * np.sqrt(252)

# Calculate Sharpe Ratio for SPY (assuming risk-free rate is 0 for simplicity)
spy_sharpe_ratio = spy_returns.mean() / spy_returns.std() * np.sqrt(252)

# Calculate Value at Risk (VaR) for SPY
# VaR at a certain confidence level (e.g., 95%) represents the maximum expected loss over a given time period.
# We'll use the historical method here.
confidence_level = 0.95
spy_VaR_95 = -np.percentile(spy_returns, 100 * (1 - confidence_level))

confidence_level_99 = 0.99
spy_VaR_99 = -np.percentile(spy_returns, 100 * (1 - confidence_level_99))


# Calculate Conditional Value at Risk (CVaR) for SPY
# CVaR (or Expected Shortfall) is the expected loss given that the loss is greater than the VaR.
# It provides a more conservative measure of risk than VaR.
# We'll calculate CVaR based on the returns that are worse than the VaR.
spy_CVaR_95 = -spy_returns[spy_returns < -spy_VaR_95].mean()
spy_CVaR_99 = -spy_returns[spy_returns < -spy_VaR_99].mean()


display(f"SPY Annualized Volatility: {spy_annualized_volatility:.4f}")
display(f"SPY Sharpe Ratio: {spy_sharpe_ratio:.4f}")
display(f"SPY VaR (95% confidence): {spy_VaR_95:.4f}")
display(f"SPY CVaR (95% confidence): {spy_CVaR_95:.4f}")
display(f"SPY VaR (99% confidence): {spy_VaR_99:.4f}")
display(f"SPY CVaR (99% confidence): {spy_CVaR_99:.4f}")

In [ ]:
import numpy as np
import pandas as pd

# Calculate Annualized Volatility (already done, but including for completeness)
annualized_volatility = portfolio_returns.std() * np.sqrt(252)

# Calculate Sharpe Ratio (already done, but including for completeness)
# Assuming risk-free rate is 0 for simplicity
sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252)

# Calculate Value at Risk (VaR)
# VaR at a certain confidence level (e.g., 95%) represents the maximum expected loss over a given time period.
# We'll use the historical method here.
confidence_level = 0.95
VaR_95 = -np.percentile(portfolio_returns, 100 * (1 - confidence_level))

confidence_level_99 = 0.99
VaR_99 = -np.percentile(portfolio_returns, 100 * (1 - confidence_level_99))


# Calculate Conditional Value at Risk (CVaR)
# CVaR (or Expected Shortfall) is the expected loss given that the loss is greater than the VaR.
# It provides a more conservative measure of risk than VaR.
# We'll calculate CVaR based on the returns that are worse than the VaR.
CVaR_95 = -portfolio_returns[portfolio_returns < -VaR_95].mean()
CVaR_99 = -portfolio_returns[portfolio_returns < -VaR_99].mean()

display(f"Portfolio Annualized Volatility (Equal Weighted): {annualized_volatility:.4f}")
display(f"Portfolio Sharpe Ratio (Equal Weighted): {sharpe_ratio:.4f}")
display(f"Portfolio VaR (95% confidence): {VaR_95:.4f}")
display(f"Portfolio CVaR (95% confidence): {CVaR_95:.4f}")
display(f"Portfolio VaR (99% confidence): {VaR_99:.4f}")
display(f"Portfolio CVaR (99% confidence): {CVaR_99:.4f}")

In [ ]:
# --- Portfolio & Data ---
PORTFOLIO_TICKERS = list(filter(lambda item: item != 'SPY', tickers))
BENCHMARK_TICKER = 'SPY'
DATA_PERIOD = '25y'

# --- Feature Engineering ---
LOG_RETURN_CLIP = 0.20 # Clip log returns to +/- 20% to remove extreme outliers

# --- Bi-LSTM Model ---
LOOKBACK_WINDOW = 60 # Days of historical data to use for one prediction
TRAIN_SPLIT = 0.64
VALIDATION_SPLIT = 0.16 # 16% validation, 20% test
LSTM_UNITS = [128, 64, 32]
DROPOUT_RATE = 0.2
EPOCHS = 100
BATCH_SIZE = 32

# --- NSGA-II Optimization ---
POPULATION_SIZE = 100 # Smaller population for more concentrated solutions
N_GENERATIONS = 200   # Fewer generations prevent over-diversification
TARGET_DIVIDEND_YIELD = 0.03 # 4% annual dividend yield
MIN_WEIGHT_PER_ASSET = 0.01 # 0% min allocation (allow zero weights for assets not selected)
MAX_WEIGHT_PER_ASSET = 0.09 # 15% max allocation (allow more concentration in top assets)

# --- Prediction Mode ---
# 'historical': Use model predictions on the test set for backtesting.
# 'future': Generate 30 new forward-looking predictions.
PREDICTION_MODE = 'historical'

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import talib as ta # Import TA-Lib

# Define the look-back period
look_back = LOOKBACK_WINDOW

# Exclude SPY from the close data for portfolio training
portfolio_close_data = close_data.drop(columns=['SPY']).copy()

print("=" * 80)
print("DATA SPLIT: Portfolio vs Benchmark")
print("=" * 80)
print(f"✓ Portfolio assets (for Bi-LSTM & NSGA-II): {len(portfolio_close_data.columns)} tickers")
print(f"  {list(portfolio_close_data.columns)}")
print(f"\n✓ Benchmark: SPY (excluded from portfolio)")
print(f"  SPY in close_data: {'SPY' in close_data.columns}")
print(f"  SPY in portfolio_close_data: {'SPY' in portfolio_close_data.columns}")
print("=" * 80)

# --- Refactor: Build ohlc_data more efficiently ---
ohlc_data_parts = []
# Track tickers that actually have OHLC data to ensure portfolio_close_data is aligned
actual_ohlc_tickers = []

for ticker in portfolio_close_data.columns:
    # Check for full OHLC data presence (Open, High, Low, Close)
    if all(col in data.columns for col in [('Open', ticker), ('High', ticker), ('Low', ticker), ('Close', ticker)]):
        temp_df = data[[('Open', ticker), ('High', ticker), ('Low', ticker), ('Close', ticker)]].copy()
        temp_df.columns = pd.MultiIndex.from_product([[ticker], ['Open', 'High', 'Low', 'Close']])
        ohlc_data_parts.append(temp_df)
        actual_ohlc_tickers.append(ticker)
    elif ('Close', ticker) in data.columns:
        # Fallback to only Close data if full OHLC is not available (e.g., for some ETFs)
        temp_df = data[[('Close', ticker)]].copy()
        temp_df.columns = pd.MultiIndex.from_product([[ticker], ['Close']])
        ohlc_data_parts.append(temp_df)
        actual_ohlc_tickers.append(ticker)
        print(f"Only Close data available for {ticker}. Proceeding with Close prices only for feature engineering.")
    else:
        # If no relevant data, print a message and this ticker won't be in actual_ohlc_tickers
        print(f"No relevant OHLC data available for {ticker}. Skipping feature engineering for this asset.")

if ohlc_data_parts:
    # Concatenate all parts into a single DataFrame
    ohlc_data = pd.concat(ohlc_data_parts, axis=1)
    # Reindex to match the original portfolio_close_data index and drop NaNs introduced by reindexing/missing data
    ohlc_data = ohlc_data.reindex(portfolio_close_data.index).dropna()
else:
    raise ValueError("No OHLC data could be constructed for any asset.")

# Adjust portfolio_close_data to only include assets for which OHLC data was successfully processed and aligned
# This ensures consistency for feature calculation base.
portfolio_close_data = portfolio_close_data[actual_ohlc_tickers].reindex(ohlc_data.index)


## Add Enhanced Lag Features and Engineered Variables

Adding comprehensive lag features similar to the polars-based approach:
- Close price lags (1, 2, 3 days)
- High-Minus-Low (HML) and its lags
- Open-Minus-Close (OMC) and its lags
- Volume lags
- Exponential moving averages with different half-lives
- Log returns based on lagged prices

In [ ]:
#--- Feature Engineering Function ---
def create_features(price_data_polars, tickers):
    """
    Generates a representative set of technical analysis features and targets
    for the given tickers, ensuring consistent date ranges and handling missing data robustly.
    """
    print(f"Generating features for {len(tickers)} tickers...")

    # Get the date range from the already processed Polars DataFrame
    start_date = price_data_polars['Date'].min()
    end_date = price_data_polars['Date'].max()

    print(f"Downloading OHLCV data from {start_date.date()} to {end_date.date()} for portfolio tickers...")
    ohlcv_data_pd = yf.download(tickers, start=start_date, end=end_date, progress=False)

    # Ensure OHLCV data has MultiIndex columns for consistency.
    if not isinstance(ohlcv_data_pd.columns, pd.MultiIndex):
        raise ValueError("Yfinance did not return MultiIndex columns for OHLCV data. Unexpected data structure.")

    # Reindex to align OHLCV data with the dates from the original `price_data_polars`
    reference_index = price_data_polars.to_pandas().set_index('Date').index
    ohlcv_data_pd = ohlcv_data_pd.reindex(reference_index)

    # Fill any NaNs after reindexing to make the data as complete as possible for feature calculations
    # First, forward fill, then backward fill to handle leading/trailing NaNs.
    # This is applied to the OHLCV data directly.
    ohlcv_data_pd = ohlcv_data_pd.ffill().bfill()

    # Drop any rows that are still entirely NaN (e.g., if a ticker was completely missing for the entire period)
    ohlcv_data_pd = ohlcv_data_pd.dropna(how='all', axis=0)

    print(f"OHLCV data shape after download, reindex, and ffill/bfill: {ohlcv_data_pd.shape}")

    # Identify tickers that have sufficient data (i.e., not entirely NaN in Close price after filling)
    clean_tickers = []
    filtered_out_tickers = []

    for ticker in tickers:
        if ('Close', ticker) not in ohlcv_data_pd.columns:
            print(f"⚠️ FILTERED: {ticker} - OHLCV data not found after processing")
            filtered_out_tickers.append((ticker, "OHLCV data not found"))
            continue
        # If Close series is still all NaN after ffill/bfill, it means no data was available for it at all.
        if ohlcv_data_pd[('Close', ticker)].isnull().all():
            print(f"⚠️ FILTERED: {ticker} - No valid Close prices after filling")
            filtered_out_tickers.append((ticker, "No valid Close prices"))
            continue

        # Check data completeness
        null_pct = ohlcv_data_pd[('Close', ticker)].isnull().sum() / len(ohlcv_data_pd) * 100
        if null_pct > 50:  # If more than 50% missing
            print(f"⚠️ FILTERED: {ticker} - {null_pct:.1f}% missing data (exceeds 50% threshold)")
            filtered_out_tickers.append((ticker, f"{null_pct:.1f}% missing data"))
            continue

        clean_tickers.append(ticker)

    if not clean_tickers:
        raise ValueError("No valid tickers remaining after filtering for excessive missing data. Cannot create features.")

    print(f"\n✅ Proceeding with {len(clean_tickers)} clean tickers out of {len(tickers)} total")
    if filtered_out_tickers:
        print(f"\n❌ Filtered out {len(filtered_out_tickers)} tickers:")
        for ticker, reason in filtered_out_tickers:
            print(f"   - {ticker}: {reason}")

    feature_dict = {}
    for ticker in clean_tickers:
        # Extract shifted OHLCV and Volume for feature calculation
        # Shift by 1 to prevent look-ahead bias.
        high = ohlcv_data_pd[('High', ticker)].shift(1)
        low = ohlcv_data_pd[('Low', ticker)].shift(1)
        close = ohlcv_data_pd[('Close', ticker)].shift(1)
        volume = ohlcv_data_pd[('Volume', ticker)].shift(1)

        # --- Momentum Indicators ---
        # TA-Lib functions handle leading NaNs by producing NaNs until enough data is available.
        feature_dict[f'{ticker}_RSI'] = talib.RSI(close)
        macd, macd_signal, macd_hist = talib.MACD(close)
        feature_dict[f'{ticker}_MACD'] = macd
        feature_dict[f'{ticker}_MACD_SIGNAL'] = macd_signal
        feature_dict[f'{ticker}_MACD_HIST'] = macd_hist
        feature_dict[f'{ticker}_ADX'] = talib.ADX(high, low, close)
        feature_dict[f'{ticker}_ROC'] = talib.ROC(close, timeperiod=LOOKBACK_WINDOW) # New: Rate of Change

        # --- Volatility Indicators ---
        feature_dict[f'{ticker}_ATR'] = talib.ATR(high, low, close)
        upper, middle, lower = talib.BBANDS(close)
        feature_dict[f'{ticker}_BB_WIDTH'] = (upper - lower) / middle

        # --- Trend Indicators ---
        feature_dict[f'{ticker}_SMA_50'] = talib.SMA(close, timeperiod=50)
        feature_dict[f'{ticker}_EMA_200'] = talib.EMA(close, timeperiod=200)

        # --- Price Lags & Volume ---
        feature_dict[f'{ticker}_VOL_SMA_20'] = talib.SMA(volume, timeperiod=20)
        for lag in [1, 5, 10, 21]:
            feature_dict[f'{ticker}_LAG_{lag}'] = close.pct_change(lag)

        # New: Rolling VWAP
        # Calculate typical price, then multiply by volume
        typical_price = (high + low + close) / 3
        typical_price_x_volume = typical_price * volume

        # Calculate rolling sums, use min_periods to get values earlier
        rolling_typical_price_x_volume_sum = typical_price_x_volume.rolling(window=LOOKBACK_WINDOW, min_periods=1).sum()
        rolling_volume_sum = volume.rolling(window=LOOKBACK_WINDOW, min_periods=1).sum()

        # Compute VWAP, handling division by zero
        vwap = rolling_typical_price_x_volume_sum.div(rolling_volume_sum.replace(0, np.nan))
        feature_dict[f'{ticker}_VWAP'] = vwap.replace([np.inf, -np.inf], np.nan) # Replace inf with NaN

        # --- Long-Term Features ---
        # 1. Long-term Volatility: 252-day rolling standard deviation of returns
        returns = close.pct_change()
        feature_dict[f'{ticker}_LT_VOL_252'] = returns.rolling(window=252, min_periods=126).std()

        # 2. Long-term Position: Price relative to 252-day and 500-day moving averages
        sma_252 = talib.SMA(close, timeperiod=252)
        sma_500 = talib.SMA(close, timeperiod=500)
        feature_dict[f'{ticker}_PRICE_TO_SMA_252'] = close / sma_252
        feature_dict[f'{ticker}_PRICE_TO_SMA_500'] = close / sma_500

        # 3. Volume Anomaly: 60-day average volume vs. 252-day average volume
        vol_sma_60 = talib.SMA(volume, timeperiod=60)
        vol_sma_252 = talib.SMA(volume, timeperiod=252)
        feature_dict[f'{ticker}_VOL_ANOMALY'] = vol_sma_60 / vol_sma_252

    # Create DataFrame efficiently from dictionary. Use the full index from ohlcv_data_pd
    features_df = pd.DataFrame(feature_dict, index=ohlcv_data_pd.index)
    # Drop columns that are entirely NaN in features_df (e.g., if a feature could not be calculated for a ticker)
    features_df = features_df.dropna(axis=1, how='all')
    print(f"features_df shape after creation and dropping all-NaN columns: {features_df.shape}")

    # --- Add VIX as a global feature ---
    print(f"Downloading VIX data (^VIX) for {start_date.date()} to {end_date.date()}...")
    vix_data_pd = yf.download('^VIX', start=start_date, end=end_date, progress=False)
    vix_data_pd = vix_data_pd['Close'].reindex(ohlcv_data_pd.index) # Align with the main OHLCV index
    vix_data_pd = vix_data_pd.ffill().bfill().shift(1) # Shift to prevent look-ahead bias and fill any NaNs
    features_df['VIX_Close'] = vix_data_pd
    print(f"features_df shape after adding VIX: {features_df.shape}")

    # --- Define Target Variable ---
    # The target is the next day's log return for each portfolio asset.
    # This is calculated on the original (non-shifted) Close price data from ohlcv_data_pd for clean_tickers.
    target_df = np.log(ohlcv_data_pd['Close'][clean_tickers] / ohlcv_data_pd['Close'][clean_tickers].shift(1)).shift(-1)
    target_df = target_df.clip(-LOG_RETURN_CLIP, LOG_RETURN_CLIP) # Clip outliers
    target_df.columns = [f'{col}_TARGET' for col in clean_tickers]
    # Drop columns that are entirely NaN in target_df
    target_df = target_df.dropna(axis=1, how='all')
    print(f"target_df shape after creation and dropping all-NaN columns: {target_df.shape}")

    # Ensure target_df only contains tickers that still have features after dropping all-NaN feature columns
    # Re-align clean_tickers based on available feature columns
    final_feature_tickers = sorted(list(set([col.split('_')[0] for col in features_df.columns if not col.endswith('_TARGET') and col != 'VIX_Close'])))
    final_target_cols = [f'{t}_TARGET' for t in final_feature_tickers if f'{t}_TARGET' in target_df.columns]
    target_df = target_df[final_target_cols] # Filter target_df to match available features
    print(f"target_df shape after aligning with features: {target_df.shape}")

    # --- Combine and clean ---
    # Align features and targets on the same index
    combined_df = pd.concat([features_df, target_df], axis=1)

    # After combining, drop rows that have any NaN, which should now only be due to
    # the initial lookback period and the final target shift.
    if combined_df.empty:
        raise ValueError("Combined DataFrame is empty after dropping all-NaN columns. Check data sources or ticker list.")

    combined_df = combined_df.dropna() # Drop rows where any value is NaN

    print(f"\u2713 Feature engineering complete. Shape: {combined_df.shape}")
    return combined_df, ohlcv_data_pd, clean_tickers

price_df = pl.from_pandas(close_data.copy().reset_index().rename(columns={'index': 'Date'}))

# --- Execute Feature Engineering ---
# We only create features for the assets in our portfolio, not the benchmark
features_and_target_df, ohlcv_data_for_cov_target, clean_tickers_for_cov_target = create_features(price_df, PORTFOLIO_TICKERS)

# Show which assets were filtered
print(f"\n📊 Asset Filtering Summary:")
print(f"   Original portfolio: {len(PORTFOLIO_TICKERS)} assets")
print(f"   After create_features: {len(clean_tickers_for_cov_target)} assets")
print(f"   Filtered out: {len(PORTFOLIO_TICKERS) - len(clean_tickers_for_cov_target)} assets")

missing_assets = set(PORTFOLIO_TICKERS) - set(clean_tickers_for_cov_target)
if missing_assets:
    print(f"\n   ❌ Missing assets after create_features: {sorted(list(missing_assets))}")
    print(f"\n   These assets were removed during feature creation due to:")
    print(f"      - Insufficient historical data (gaps, missing OHLCV)")
    print(f"      - Forward/backward fill couldn't reconstruct complete series")
    print(f"      - Failed technical indicator calculations")

    # Check if the user's 8 assets of interest are in the missing list
    user_interest_assets = ['NVDA', 'AMZN', 'AMD', 'INTC', 'META', 'AVGO', 'XOM', 'VDE']
    missing_from_interest = [a for a in user_interest_assets if a in missing_assets]
    if missing_from_interest:
        print(f"\n   📌 From your interest list, these {len(missing_from_interest)} were filtered:")
        print(f"      {missing_from_interest}")
else:
    print(f"\n   ✅ No assets filtered (all {len(PORTFOLIO_TICKERS)} retained)")

print("\nFeatures and Target Data Tail:")
display(features_and_target_df.tail(3))

In [ ]:
def generate_covariance_targets(ohlcv_data_pd, tickers, window_size=LOOKBACK_WINDOW):
    """
    Generates future realized covariance matrices as targets for each date.
    For each date, it calculates the covariance of daily log returns over the next `window_size` days.
    """
    print(f"Generating {window_size}-day future covariance targets for {len(tickers)} tickers...")

    # Calculate log returns for all tickers
    log_returns = np.log(ohlcv_data_pd['Close'][tickers] / ohlcv_data_pd['Close'][tickers].shift(1))
    log_returns = log_returns.clip(-LOG_RETURN_CLIP, LOG_RETURN_CLIP) # Clip outliers
    log_returns = log_returns.dropna() # Drop initial NaNs from shift(1)

    covariance_targets = []
    target_dates = []

    # The features_and_target_df.index represents the date 't' for which we are predicting
    # the returns for 't+1' and covariance for 't+1' to 't+1+window_size-1'.
    # So we need to look for future returns starting from the day *after* the current_date.
    for i in range(len(features_and_target_df.index)):
        current_date_for_prediction = features_and_target_df.index[i]

        # Find the starting index for the future covariance period in `log_returns`.
        # This should be the first entry *after* `current_date_for_prediction`.
        start_search_idx = log_returns.index.searchsorted(current_date_for_prediction)

        # If searchsorted finds an exact match for current_date_for_prediction, we need to advance one day.
        # Otherwise, start_search_idx already points to the first date strictly greater than current_date_for_prediction.
        if start_search_idx < len(log_returns) and log_returns.index[start_search_idx] == current_date_for_prediction:
            start_cov_period_idx = start_search_idx + 1
        else:
            start_cov_period_idx = start_search_idx

        # Check if there are enough future data points from this starting index
        if start_cov_period_idx + window_size > len(log_returns):
            covariance_targets.append(np.full(len(tickers) * (len(tickers) + 1) // 2, np.nan)) # Fill with NaNs
            target_dates.append(current_date_for_prediction)
            continue

        future_returns_slice = log_returns.iloc[start_cov_period_idx : start_cov_period_idx + window_size]

        # Ensure enough non-NaN data points to calculate covariance within the slice
        # If any of the future_returns_slice has NaNs, or if it's not the full window_size, skip
        if future_returns_slice.isnull().any().any() or len(future_returns_slice) < window_size:
            covariance_targets.append(np.full(len(tickers) * (len(tickers) + 1) // 2, np.nan))
        else:
            # Calculate covariance matrix
            cov_matrix = future_returns_slice.cov()
            # Flatten the upper triangular part of the matrix (including diagonal)
            # to avoid redundancy and keep target size manageable
            # Number of unique elements in a symmetric matrix (n*n) is n*(n+1)/2
            flat_cov = cov_matrix.values[np.triu_indices(len(tickers))]
            covariance_targets.append(flat_cov)

        target_dates.append(current_date_for_prediction)

    # Create a DataFrame for the covariance targets
    num_cov_elements = len(tickers) * (len(tickers) + 1) // 2
    cov_target_df = pd.DataFrame(covariance_targets, index=target_dates,
                                 columns=[f'COV_TARGET_{i}' for i in range(num_cov_elements)])

    print(f"\u2713 Covariance targets generated. Shape: {cov_target_df.shape}")
    return cov_target_df


In [ ]:
print("Regenerating covariance targets and combining with features...")

# 1. Call the `generate_covariance_targets` function
covariance_target_df = generate_covariance_targets(ohlcv_data_for_cov_target, clean_tickers_for_cov_target)

# 2. Get the clean feature columns (excluding any targets, old or new)
feature_cols_only = [c for c in features_and_target_df.columns if not (c.endswith('_TARGET') or c.startswith('COV_TARGET_'))]
features_only_df_clean = features_and_target_df[feature_cols_only]

# 3. Get the clean return target columns (excluding covariance targets)
return_target_cols_clean = [c for c in features_and_target_df.columns if c.endswith('_TARGET') and not c.startswith('COV_TARGET_')]
return_targets_df_clean = features_and_target_df[return_target_cols_clean]

# Diagnostic: Check how many tickers have return targets
print(f"\n🔍 Return Target Diagnostic:")
print(f"   Number of return target columns: {len(return_target_cols_clean)}")
print(f"   Expected from clean_tickers: {len(clean_tickers_for_cov_target)}")
if len(return_target_cols_clean) != len(clean_tickers_for_cov_target):
    print(f"   ⚠️ MISMATCH: {len(clean_tickers_for_cov_target) - len(return_target_cols_clean)} targets missing!")
    expected_targets = [f'{ticker}_TARGET' for ticker in clean_tickers_for_cov_target]
    actual_targets = set(return_target_cols_clean)
    missing_targets = [t for t in expected_targets if t not in actual_targets]
    if missing_targets:
        missing_tickers = [t.replace('_TARGET', '') for t in missing_targets]
        print(f"   ❌ Missing return targets for: {missing_tickers}")
else:
    print(f"   ✅ All {len(return_target_cols_clean)} return targets present")

# 4. Reindex covariance_target_df to match the dates of clean features for consistent merging
covariance_target_df = covariance_target_df.reindex(features_only_df_clean.index)

# 5. Combine features, original return targets, and the *new* covariance targets cleanly
features_and_all_targets_df = pd.concat([
    features_only_df_clean,
    return_targets_df_clean,
    covariance_target_df
], axis=1)

# 6. Drop any rows with NaN values from `features_and_all_targets_df`
# This ensures that each sample has complete feature and target information.
rows_before_dropna = len(features_and_all_targets_df)
features_and_all_targets_df = features_and_all_targets_df.dropna()
rows_after_dropna = len(features_and_all_targets_df)

print(f"\n🔍 NaN Handling Diagnostic:")
print(f"   Rows before dropna: {rows_before_dropna}")
print(f"   Rows after dropna: {rows_after_dropna}")
print(f"   Rows dropped: {rows_before_dropna - rows_after_dropna}")

# Check if any columns (assets) were completely removed
columns_before = set(return_target_cols_clean)
# After dropna, check which return targets still have any non-null values
remaining_return_targets = [c for c in return_target_cols_clean if c in features_and_all_targets_df.columns and features_and_all_targets_df[c].notna().any()]
columns_after = set(remaining_return_targets)
removed_columns = columns_before - columns_after

if removed_columns:
    removed_tickers = [c.replace('_TARGET', '') for c in removed_columns]
    print(f"\n   ⚠️ ASSET REMOVAL DETECTED!")
    print(f"   {len(removed_tickers)} assets removed after dropna: {removed_tickers}")

    # Check if these match the user's interest list
    user_interest_assets = ['NVDA', 'AMZN', 'AMD', 'INTC', 'META', 'AVGO', 'XOM', 'VDE']
    removed_from_interest = [t for t in removed_tickers if t in user_interest_assets]
    if removed_from_interest:
        print(f"   📌 From your interest list: {removed_from_interest}")
else:
    print(f"   ✅ No assets removed (all {len(columns_before)} retained)")

# 7. Update the global variable `features_and_target_df`
global features_and_target_df
features_and_target_df = features_and_all_targets_df

# 8. Redefine `target_cols` based on the newly constructed `features_and_target_df`
global target_cols
target_cols = [col for col in features_and_target_df.columns if col.endswith('_TARGET') or col.startswith('COV_TARGET_')]

# 9. Update the global variable `output_shape`
global output_shape
output_shape = len(target_cols)

# 10. Print the shape of the updated `features_and_target_df` and the count of `target_cols`
print(f"Updated features_and_target_df shape: {features_and_target_df.shape}")
print(f"Updated target_cols count: {len(target_cols)}")

# 11. Display the tail of the updated `features_and_target_df`
print("\nUpdated Features and Target Data Tail:")
display(features_and_target_df.tail(3))

In [ ]:
print("Preparing data for LSTM model...")

# Separate feature columns from target columns
feature_cols = [c for c in features_and_target_df.columns if not (c.endswith('_TARGET') or c.startswith('COV_TARGET_'))]

# Identify return target columns and covariance target columns
return_target_cols = [c for c in features_and_target_df.columns if c.endswith('_TARGET') and not c.startswith('COV_TARGET_')]
cov_target_cols = [c for c in features_and_target_df.columns if c.startswith('COV_TARGET_')]

# Determine the number of targets
num_return_targets = len(return_target_cols)
num_cov_targets = len(cov_target_cols)
print(f"Number of return targets: {num_return_targets}")
print(f"Number of covariance targets: {num_cov_targets}")

# Extract features and targets
X = features_and_target_df[feature_cols].values
y_returns = features_and_target_df[return_target_cols].values
y_cov = features_and_target_df[cov_target_cols].values

# Scale features and targets separately
feature_scaler = MinMaxScaler(feature_range=(-1, 1))
returns_scaler = MinMaxScaler(feature_range=(-1, 1)) # Scaler for return targets
cov_scaler = MinMaxScaler(feature_range=(-1, 1))     # Scaler for covariance targets

X_scaled = feature_scaler.fit_transform(X)
y_returns_scaled = returns_scaler.fit_transform(y_returns)
y_cov_scaled = cov_scaler.fit_transform(y_cov)

# --- 2. Create Sequences ---
def create_sequences(X_data, y_returns_data, y_cov_data, window_size):
    X_seq, y_returns_seq, y_cov_seq = [], [], []
    for i in range(len(X_data) - window_size):
        X_seq.append(X_data[i:(i + window_size)])
        y_returns_seq.append(y_returns_data[i + window_size])
        y_cov_seq.append(y_cov_data[i + window_size])
    return np.array(X_seq), np.array(y_returns_seq), np.array(y_cov_seq)

X_seq, y_returns_seq, y_cov_seq = create_sequences(X_scaled, y_returns_scaled, y_cov_scaled, LOOKBACK_WINDOW)

# --- 3. Train/Val/Test Split (Temporal) ---
n_samples = len(X_seq)
train_end = int(n_samples * TRAIN_SPLIT)
val_end = int(n_samples * (TRAIN_SPLIT + VALIDATION_SPLIT))

X_train, y_train_returns, y_train_cov = X_seq[:train_end], y_returns_seq[:train_end], y_cov_seq[:train_end]
X_val, y_val_returns, y_val_cov = X_seq[train_end:val_end], y_returns_seq[train_end:val_end], y_cov_seq[train_end:val_end]
X_test, y_test_returns, y_test_cov = X_seq[val_end:], y_returns_seq[val_end:], y_cov_seq[val_end:]

print(f"X_train shape: {X_train.shape}, y_train_returns shape: {y_train_returns.shape}, y_train_cov shape: {y_train_cov.shape}")
print(f"X_val shape: {X_val.shape}, y_val_returns shape: {y_val_returns.shape}, y_val_cov shape: {y_val_cov.shape}")
print(f"X_test shape: {X_test.shape}, y_test_returns shape: {y_test_returns.shape}, y_test_cov shape: {y_test_cov.shape}")

# --- 4. Model Architecture & Training (Modified) ---
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

@tf.keras.utils.register_keras_serializable()
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at
        return K.sum(output, axis=1)

input_shape = (X_train.shape[1], X_train.shape[2])

def create_advanced_bilstm_model(input_shape, num_return_targets, num_cov_targets, lstm_units, dropout_rate):
    inputs = tf.keras.Input(shape=input_shape)
    x = Bidirectional(LSTM(lstm_units[0], return_sequences=True))(inputs)
    x = Dropout(dropout_rate)(x)
    x = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(x)
    x = Dropout(dropout_rate)(x)
    attention_output = Attention()(x)
    x = Dropout(dropout_rate)(attention_output)

    # Output for returns
    returns_output = Dense(num_return_targets, activation='tanh', name='returns_output')(x)

    # Output for covariance
    covariance_output = Dense(num_cov_targets, activation='linear', name='covariance_output')(x)

    model = tf.keras.Model(inputs=inputs, outputs=[returns_output, covariance_output])
    model.compile(optimizer='adam', loss={'returns_output': Huber(), 'covariance_output': 'mean_squared_error'})
    return model

model = create_advanced_bilstm_model(input_shape, num_return_targets, num_cov_targets, LSTM_UNITS, DROPOUT_RATE)
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('bilstm_best_model.keras', save_best_only=True, monitor='val_loss')

print("\n--- Starting Model Training ---")
history = model.fit(
    X_train,
    {'returns_output': y_train_returns, 'covariance_output': y_train_cov},
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, {'returns_output': y_val_returns, 'covariance_output': y_val_cov}),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)
print("--- Model Training Complete ---")

# --- 5. Plot Training History ---
plt.figure(figsize=(10, 6))
plt.plot(history.history['returns_output_loss'], label='Training Returns Loss')
plt.plot(history.history['val_returns_output_loss'], label='Validation Returns Loss')
plt.plot(history.history['covariance_output_loss'], label='Training Covariance Loss')
plt.plot(history.history['val_covariance_output_loss'], label='Validation Covariance Loss')
plt.title('Model Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

**Reasoning**:
Use the trained Bi-LSTM model to make predictions on the test data.

### Classification Analysis: Direction Prediction Accuracy

Convert continuous return predictions to binary up/down signals and evaluate classification performance with confusion matrices.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

# Get predictions on test set - model has multiple outputs
predictions = model.predict(X_test)

# Extract returns predictions (first output)
if isinstance(predictions, list):
    y_pred_returns_scaled = predictions[0]  # Returns are the first output
else:
    y_pred_returns_scaled = predictions

# Inverse transform to get actual return values
y_pred_returns = returns_scaler.inverse_transform(y_pred_returns_scaled)
y_actual_returns = returns_scaler.inverse_transform(y_test_returns)

# Get the correct number of assets
num_assets = y_pred_returns.shape[1]
asset_tickers = portfolio_close_data.columns[:num_assets]

# Convert to DataFrame for easier manipulation
y_pred_df = pd.DataFrame(y_pred_returns, columns=asset_tickers)
y_actual_df = pd.DataFrame(y_actual_returns, columns=asset_tickers)

print(f"Predictions shape: {y_pred_returns.shape}")
print(f"Actual shape: {y_actual_returns.shape}")
print(f"Number of assets: {num_assets}")
print(f"Asset tickers: {list(asset_tickers)}")

# Convert continuous returns to binary signals (1 = Up/Positive, 0 = Down/Negative)
y_pred_signals = (y_pred_df > 0).astype(int)
y_actual_signals = (y_actual_df > 0).astype(int)

print("\n" + "=" * 80)
print("DIRECTIONAL PREDICTION ACCURACY - CLASSIFICATION ANALYSIS")
print("=" * 80)
print("\nConverting continuous returns to binary up/down signals:")
print("  • Signal = 1 (UP): Predicted/Actual return > 0")
print("  • Signal = 0 (DOWN): Predicted/Actual return ≤ 0")
print("\n" + "=" * 80)

# Calculate metrics for each asset
accuracy_scores = {}
for ticker in asset_tickers:
    accuracy = accuracy_score(y_actual_signals[ticker], y_pred_signals[ticker])
    accuracy_scores[ticker] = accuracy

# Overall accuracy across all predictions
overall_accuracy = accuracy_score(y_actual_signals.values.flatten(),
                                  y_pred_signals.values.flatten())

print(f"\n{'Asset':<10} {'Directional Accuracy':>20}")
print("-" * 35)
for ticker, acc in sorted(accuracy_scores.items(), key=lambda x: x[1], reverse=True):
    print(f"{ticker:<10} {acc:>19.2%}")
print("-" * 35)
print(f"{'OVERALL':<10} {overall_accuracy:>19.2%}")

# Create confusion matrices for selected assets (top 6 by accuracy)
top_assets = sorted(accuracy_scores.items(), key=lambda x: x[1], reverse=True)[:6]
top_asset_tickers = [t[0] for t in top_assets]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Confusion Matrices: Direction Prediction (UP vs DOWN)',
             fontsize=16, fontweight='bold', y=1.00)

for idx, ticker in enumerate(top_asset_tickers):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]

    # Calculate confusion matrix
    cm = confusion_matrix(y_actual_signals[ticker], y_pred_signals[ticker])

    # Plot heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['DOWN (0)', 'UP (1)'],
                yticklabels=['DOWN (0)', 'UP (1)'],
                cbar=True, ax=ax, annot_kws={'size': 14})

    ax.set_xlabel('Predicted', fontsize=11, fontweight='bold')
    ax.set_ylabel('Actual', fontsize=11, fontweight='bold')
    ax.set_title(f'{ticker} - Accuracy: {accuracy_scores[ticker]:.2%}',
                fontsize=12, fontweight='bold', pad=10)

plt.tight_layout()
plt.show()

# Detailed classification report for overall performance
print("\n" + "=" * 80)
print("OVERALL CLASSIFICATION REPORT")
print("=" * 80)
print("\nAggregated across all assets and time steps:\n")
print(classification_report(y_actual_signals.values.flatten(),
                          y_pred_signals.values.flatten(),
                          target_names=['DOWN (0)', 'UP (1)'],
                          digits=4))

# Overall confusion matrix
print("\n" + "=" * 80)
print("OVERALL CONFUSION MATRIX")
print("=" * 80)
cm_overall = confusion_matrix(y_actual_signals.values.flatten(),
                              y_pred_signals.values.flatten())

plt.figure(figsize=(8, 6))
sns.heatmap(cm_overall, annot=True, fmt='d', cmap='YlGnBu',
            xticklabels=['DOWN (0)', 'UP (1)'],
            yticklabels=['DOWN (0)', 'UP (1)'],
            cbar_kws={'label': 'Count'},
            annot_kws={'size': 16, 'fontweight': 'bold'})
plt.xlabel('Predicted Direction', fontsize=13, fontweight='bold')
plt.ylabel('Actual Direction', fontsize=13, fontweight='bold')
plt.title(f'Overall Confusion Matrix\nAccuracy: {overall_accuracy:.2%}',
         fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

# Calculate precision for UP predictions (important for trading)
tn, fp, fn, tp = cm_overall.ravel()
precision_up = tp / (tp + fp) if (tp + fp) > 0 else 0
recall_up = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_up = 2 * (precision_up * recall_up) / (precision_up + recall_up) if (precision_up + recall_up) > 0 else 0

print("\n" + "=" * 80)
print("KEY METRICS FOR UP PREDICTIONS (Trading Signals)")
print("=" * 80)
print(f"Precision (UP): {precision_up:.2%}  →  When model predicts UP, it's correct {precision_up:.2%} of time")
print(f"Recall (UP):    {recall_up:.2%}  →  Model captures {recall_up:.2%} of actual UP movements")
print(f"F1-Score (UP):  {f1_up:.2%}  →  Harmonic mean of precision and recall")
print("=" * 80)

In [ ]:
print("Generating predictions from the trained Bi-LSTM model...")
# Load the best model saved during training
best_model = tf.keras.models.load_model('bilstm_best_model.keras', custom_objects={'Huber': Huber, 'Attention': Attention})

# Predict on the test set, which now returns two outputs
predicted_returns_scaled, predicted_covariances_scaled = best_model.predict(X_test)

# Inverse transform the predictions for returns and covariances
predicted_log_returns = returns_scaler.inverse_transform(predicted_returns_scaled)
predicted_covariances_flattened = cov_scaler.inverse_transform(predicted_covariances_scaled)

# Convert log returns to simple returns
predicted_simple_returns = np.expm1(predicted_log_returns)

# Create DataFrame for predicted simple returns
predicted_returns_df = pd.DataFrame(predicted_simple_returns, columns=return_target_cols)

print(f"✓ Predicted returns generated. Shape: {predicted_returns_df.shape}")
print(f"✓ Predicted flattened covariances generated. Shape: {predicted_covariances_flattened.shape}")

In [ ]:
def flattened_to_cov_matrix(flattened_cov, n_assets):
    """
    Reconstructs a full symmetric covariance matrix from its flattened upper triangular form.
    """
    cov_matrix = np.zeros((n_assets, n_assets))
    # Get indices for the upper triangle (including diagonal)
    triu_indices = np.triu_indices(n_assets)
    # Fill the upper triangle
    cov_matrix[triu_indices] = flattened_cov
    # Fill the lower triangle to make it symmetric
    cov_matrix = cov_matrix + cov_matrix.T - np.diag(np.diag(cov_matrix))
    return cov_matrix

# Apply the helper function to each predicted flattened covariance
n_assets = len(PORTFOLIO_TICKERS)
predicted_cov_matrices_time_series = [
    flattened_to_cov_matrix(row, n_assets) for row in predicted_covariances_flattened
]

print(f"✓ Generated {len(predicted_cov_matrices_time_series)} predicted covariance matrices.")
print(f"Example of a predicted covariance matrix shape: {predicted_cov_matrices_time_series[0].shape}")

## Define the optimization problem

### Subtask:
Clearly define the objectives (e.g., maximize predicted return, maximize predicted dividend yield, minimize predicted risk - volatility/CVaR) and constraints (e.g., sum of weights equals 1, minimum/maximum allocation per asset) for the portfolio optimization using NSGA-II.


**Reasoning**:
Document the objectives and constraints for the NSGA-II algorithm based on the task description and the referenced paper's likely approach to multi-objective portfolio optimization.



In [ ]:
# Objectives for NSGA-II:
# 1. Maximize Predicted Portfolio Return: Based on the price predictions from the Bi-LSTM model.
# 2. Maximize Predicted Portfolio Dividend Yield: Based on the predicted dividends and predicted prices.
# 3. Minimize Predicted Portfolio Risk: This can be measured using predicted portfolio volatility or Conditional Value at Risk (CVaR) calculated from the predicted returns.

# Constraints for NSGA-II:
# 1. Sum of Weights: The sum of the weights of all assets in the portfolio must equal 1. (Σ wi = 1)
# 2. Weight Bounds: Define minimum and maximum allocation percentages for each asset (e.g., 0 <= wi <= 1, or with more specific bounds like 0.05 <= wi <= 0.4).
# 3. Target Dividend Yield (Implicit): The NSGA-II will aim to find solutions on the Pareto front that satisfy the 4% dividend return requirement. This might be handled as a soft constraint or by selecting solutions from the Pareto front that meet this criterion.

print("NSGA-II Objectives:")
print("1. Maximize Predicted Portfolio Return")
print("2. Maximize Predicted Portfolio Dividend Yield")
print("3. Minimize Predicted Portfolio Risk (Volatility/CVaR)")
print("\nNSGA-II Constraints:")
print("1. Sum of Weights = 1")
print("2. Weight Bounds (e.g., 0 <= wi <= 1)")
print("3. Target Dividend Yield (Implicit objective for solution selection)")

## Prepare input data for nsga-ii

### Subtask:
Prepare the predicted stock movements and dividend information in a format suitable for the NSGA-II algorithm. This might involve using the predictions from the Bi-LSTM model.


## 🎯 Simplified Pipeline: Bi-LSTM → NSGA-II

**Original Complex Approach:**
1. Model predicts log returns ✓
2. Convert log returns → prices
3. Compare with previous actual prices
4. Calculate returns from price differences
5. Run Monte Carlo simulation (unused)
6. Pass returns to NSGA-II

**New Simplified Approach:**
1. Model predicts log returns ✓
2. Convert to simple returns: `r = exp(log_r) - 1` ✓
3. Pass directly to NSGA-II ✓

**Benefits:**
- ✅ Fewer transformation steps = less error accumulation
- ✅ No dependency on previous actual prices (avoids look-ahead issues)
- ✅ Faster execution (no unnecessary Monte Carlo)
- ✅ Direct connection: model output → optimizer input
- ✅ Cleaner code and easier to debug

**What NSGA-II receives:**
- DataFrame of predicted daily returns (simple returns)
- Shape: (n_test_days, n_assets)
- Used to calculate portfolio return, volatility, and Sharpe ratio

## Prepare Predicted Returns for NSGA-II

**Simplified Approach:**
Since the Bi-LSTM model now predicts log returns (not prices), we can directly use those predictions for NSGA-II optimization. This eliminates the need to:
- Convert predictions back to prices
- Compare with previous actual prices
- Recalculate returns

**Process:**
1. Take predicted log returns from model
2. Convert to simple returns: `r = exp(log_r) - 1`
3. Pass directly to NSGA-II

This is more direct, avoids compounding errors, and maintains consistency with the model's output.

## 🎯 Configure NSGA-II Input: Choose Prediction Mode

You can now choose which predictions to use for portfolio optimization:

**Option 1: Historical Test Set Predictions** (already available)
- Uses the test set period (e.g., 2023-03 to 2025-09)
- Large sample size (~600+ days)
- Can be validated against actual returns
- Good for backtesting and model evaluation

**Option 2: 30-Day Forward Predictions** (new capability)
- Predicts 30 days into the future from today
- Smaller sample (30 days)
- Forward-looking portfolio optimization
- Best for real-world trading decisions

**Which to use?**
- For **backtesting and validation**: Use historical test predictions
- For **actual portfolio selection**: Use 30-day forward predictions
- For **research**: Compare both approaches

## Run the NSGA-II Optimization

### Subtask:
Execute the NSGA-II algorithm to find a set of non-dominated portfolios (the Pareto front).

**Reasoning**:
Instantiate the portfolio optimization problem and run the NSGA-II algorithm from pymoo.

In [ ]:
print("Updating PortfolioOptimizationProblem class...")

# Extract test_dates corresponding to the X_test data points
# X_seq, y_seq are created from features_and_target_df starting from index 0
# X_test starts from index `val_end` of X_seq
# So, the dates for X_test start from `features_and_target_df.index[val_end + LOOKBACK_WINDOW]`
# and for each sample in X_test, the corresponding date is its index.

# The original features_and_target_df has one entry for each date, and X_seq was built from it.
# The first `LOOKBACK_WINDOW` rows were effectively used to create the first sequence, so the first
# prediction corresponds to the date at index `LOOKBACK_WINDOW`.
# X_test starts at index `val_end` of X_seq. The date for the first sample in X_test is at
# `features_and_target_df.index[val_end + LOOKBACK_WINDOW]`.

test_dates = features_and_target_df.index[val_end + LOOKBACK_WINDOW : val_end + LOOKBACK_WINDOW + len(X_test)]

class PortfolioOptimizationProblem(Problem):
    def __init__(self, mu, dividend_yields, predicted_returns_timeseries,
                 predicted_cov_matrices_time_series, test_dates,
                 min_weight, max_weight, target_dividend, max_portfolio_volatility=None, min_portfolio_return=None):

        # Add 4th constraint if min_portfolio_return is specified
        n_constraints = 2
        if max_portfolio_volatility is not None:
            n_constraints += 1
        if min_portfolio_return is not None:
            n_constraints += 1

        super().__init__(n_var=len(mu),
                         n_obj=4,
                         n_constr=n_constraints, # sum of weights = 1, dividend yield >= target, [optional] volatility <= max, [optional] return >= min
                         xl=min_weight,
                         xu=max_weight)
        self.mu = mu
        self.dividend_yields = dividend_yields
        self.predicted_returns_timeseries = predicted_returns_timeseries
        self.predicted_cov_matrices_time_series = predicted_cov_matrices_time_series
        self.test_dates = test_dates # Index for mapping predicted covariance matrices
        self.target_dividend = target_dividend
        self.max_portfolio_volatility = max_portfolio_volatility
        self.min_portfolio_return = min_portfolio_return

    def _evaluate(self, x, out, *args, **kwargs):
        # Normalize weights to ensure they sum to 1
        weights = x / x.sum(axis=1, keepdims=True)

        # Initialize objectives arrays
        f1 = np.zeros(x.shape[0]) # Negative Return
        f2 = np.zeros(x.shape[0]) # Negative Dividend Yield
        f3 = np.zeros(x.shape[0]) # Volatility
        f4 = np.zeros(x.shape[0]) # CVaR

        # Initialize constraints arrays
        g1 = np.zeros(x.shape[0]) # Sum of weights
        g2 = np.zeros(x.shape[0]) # Dividend yield constraint
        constraint_idx = 3
        if self.max_portfolio_volatility is not None:
            g3 = np.zeros(x.shape[0]) # Maximum volatility constraint
            constraint_idx += 1
        if self.min_portfolio_return is not None:
            g_return = np.zeros(x.shape[0]) # Minimum return constraint

        for i, current_weights in enumerate(weights):
            # Use the predicted covariance matrix that corresponds to the current test period
            # The `mu` and `predicted_cov_matrices_time_series` are aligned with the test set.
            # The `predicted_cov_matrices_time_series` is already a list indexed 0 to len(X_test)-1.
            # So we can directly use `i` as the index.
            current_sigma = self.predicted_cov_matrices_time_series[i % len(self.predicted_cov_matrices_time_series)]

            # 1. Maximize Annualized Return (minimize negative return)
            annualized_return = np.dot(current_weights, self.mu) * 252
            f1[i] = -annualized_return

            # 2. Maximize Dividend Yield (minimize negative yield)
            f2[i] = -np.sum(current_weights * self.dividend_yields)

            # 3. Minimize Annualized Volatility
            portfolio_volatility = np.sqrt(np.dot(current_weights.T, np.dot(current_sigma, current_weights)))
            annualized_volatility = portfolio_volatility * np.sqrt(252)
            f3[i] = annualized_volatility

            # 4. Minimize CVaR (95%)
            # CVaR is calculated using the predicted returns timeseries for the portfolio
            ts_idx = i % len(self.predicted_returns_timeseries)
            portfolio_returns_ts = (self.predicted_returns_timeseries[ts_idx] @ current_weights)
            q = np.percentile(portfolio_returns_ts, 5)
            cvar = np.mean(portfolio_returns_ts[portfolio_returns_ts <= q])
            f4[i] = -cvar * 252 # Minimize CVaR (less negative is better)

            # --- Constraints ---
            # Constraint 1: sum of weights must be 1 (pymoo handles this as g <= 0)
            # Since we normalize weights above, this should always be ~0, but we keep small tolerance
            g1[i] = np.abs(np.sum(current_weights) - 1.0) - 1e-6  # Allow small numerical error

            # Constraint 2: portfolio dividend yield must be >= target
            # g <= 0 means: target - actual <= 0, or actual >= target
            portfolio_yield = np.sum(current_weights * self.dividend_yields)
            g2[i] = self.target_dividend - portfolio_yield

            # Constraint 3: portfolio volatility must be <= max_portfolio_volatility
            # g <= 0 means: actual - max <= 0, or actual <= max
            if self.max_portfolio_volatility is not None:
                g3[i] = annualized_volatility - self.max_portfolio_volatility

            # Constraint 4: portfolio return must be >= min_portfolio_return
            # g <= 0 means: min - actual <= 0, or actual >= min
            if self.min_portfolio_return is not None:
                g_return[i] = self.min_portfolio_return - annualized_return

        out["F"] = np.column_stack([f1, f2, f3, f4])

        # Build constraints dynamically
        constraints = [g1, g2]
        if self.max_portfolio_volatility is not None:
            constraints.append(g3)
        if self.min_portfolio_return is not None:
            constraints.append(g_return)
        out["G"] = np.column_stack(constraints)

print("✓ PortfolioOptimizationProblem class updated.")

In [ ]:
# --- Run NSGA-II Optimization ---
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

print("Running NSGA-II optimization...")

# Create the optimization problem with constraints
problem = PortfolioOptimizationProblem(
    mu=mu,
    dividend_yields=avg_annual_yield,
    predicted_returns_timeseries=predicted_simple_returns,
    predicted_cov_matrices_time_series=predicted_cov_matrices_time_series,
    test_dates=test_dates,
    min_weight=MIN_WEIGHT_PER_ASSET,
    max_weight=MAX_WEIGHT_PER_ASSET,
    target_dividend=TARGET_DIVIDEND_YIELD,
    max_portfolio_volatility=0.22,  # 22% max portfolio volatility
    min_portfolio_return=0.09  # 9% minimum return
)

# Configure NSGA-II algorithm
algorithm = NSGA2(
    pop_size=POPULATION_SIZE,
    eliminate_duplicates=True
)

# Run optimization
res = minimize(
    problem,
    algorithm,
    ('n_gen', N_GENERATIONS),
    seed=42,
    verbose=True
)

# --- Process Results ---
print(f"\n✓ Optimization complete!")
print(f"   Found {len(res.F)} solutions on the Pareto front")

# Extract solutions
if hasattr(res, 'F') and res.F is not None:
    # Get solutions from Pareto front
    solutions = res.X
    objectives = res.F
    print(f"   Using Pareto front: {len(solutions)} solutions")
else:
    # Fallback to population
    solutions = res.pop.get("X")
    objectives = res.pop.get("F")
    print(f"   Using population: {len(solutions)} solutions")

# Normalize weights (ensure they sum to 1)
weight_sums = solutions.sum(axis=1, keepdims=True)
weight_sums = np.where(weight_sums == 0, 1, weight_sums)  # Avoid division by zero
solutions_normalized = solutions / weight_sums

# Create results DataFrame
pareto_results = pd.DataFrame(solutions_normalized, columns=asset_tickers)
pareto_results['Return'] = -objectives[:, 0]  # Negate because we minimized
pareto_results['Dividend'] = -objectives[:, 1]  # Negate because we minimized
pareto_results['Volatility'] = objectives[:, 2]
pareto_results['CVaR'] = objectives[:, 3]
pareto_results['Sharpe'] = (pareto_results['Return'] - 0.02) / pareto_results['Volatility']

print("\nPareto Front Summary:")
print(f"Return range: [{pareto_results['Return'].min():.4f}, {pareto_results['Return'].max():.4f}]")
print(f"Dividend range: [{pareto_results['Dividend'].min():.4f}, {pareto_results['Dividend'].max():.4f}]")
print(f"Volatility range: [{pareto_results['Volatility'].min():.4f}, {pareto_results['Volatility'].max():.4f}]")
print(f"CVaR range: [{pareto_results['CVaR'].min():.4f}, {pareto_results['CVaR'].max():.4f}]")
print(f"Sharpe range: [{pareto_results['Sharpe'].min():.4f}, {pareto_results['Sharpe'].max():.4f}]")

# --- Visualize Pareto Front ---
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Return vs Volatility (colored by Sharpe)
scatter1 = axes[0].scatter(
    pareto_results['Volatility']*100,
    pareto_results['Return']*100,
    c=pareto_results['Sharpe'],
    cmap='viridis',
    s=100,
    alpha=0.6,
    edgecolors='black'
)
# Highlight highest Sharpe
best_sharpe_idx = pareto_results['Sharpe'].idxmax()
axes[0].scatter(
    pareto_results.loc[best_sharpe_idx, 'Volatility']*100,
    pareto_results.loc[best_sharpe_idx, 'Return']*100,
    c='red',
    s=300,
    marker='*',
    edgecolors='black',
    linewidths=2,
    label='Highest Sharpe',
    zorder=5
)
axes[0].set_xlabel('Annualized Volatility (%)', fontsize=12)
axes[0].set_ylabel('Expected Annual Return (%)', fontsize=12)
axes[0].set_title('Efficient Frontier: Return vs Volatility', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Sharpe Ratio')

# Plot 2: Return vs Dividend (colored by Volatility)
scatter2 = axes[1].scatter(
    pareto_results['Dividend']*100,
    pareto_results['Return']*100,
    c=pareto_results['Volatility']*100,
    cmap='coolwarm',
    s=100,
    alpha=0.6,
    edgecolors='black'
)
# Highlight highest Sharpe
axes[1].scatter(
    pareto_results.loc[best_sharpe_idx, 'Dividend']*100,
    pareto_results.loc[best_sharpe_idx, 'Return']*100,
    c='red',
    s=300,
    marker='*',
    edgecolors='black',
    linewidths=2,
    label='Highest Sharpe',
    zorder=5
)
axes[1].set_xlabel('Dividend Yield (%)', fontsize=12)
axes[1].set_ylabel('Expected Annual Return (%)', fontsize=12)
axes[1].set_title('Return vs Dividend Yield', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Volatility (%)')

plt.tight_layout()
plt.show()

print("\n✓ Visualization complete!")
print(f"   Best Sharpe Ratio: {pareto_results.loc[best_sharpe_idx, 'Sharpe']:.4f}")
print(f"   Return: {pareto_results.loc[best_sharpe_idx, 'Return']:.4%}")
print(f"   Volatility: {pareto_results.loc[best_sharpe_idx, 'Volatility']:.4%}")
print(f"   Dividend: {pareto_results.loc[best_sharpe_idx, 'Dividend']:.4%}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Select the Best Portfolio for Backtesting ---
print("Selecting portfolio with the highest Sharpe Ratio for backtesting...")
best_sharpe_idx = pareto_results['Sharpe'].idxmax()
selected_portfolio = pareto_results.loc[best_sharpe_idx]
selected_weights = selected_portfolio[asset_tickers]

print("\nSelected Portfolio Weights (allocations > .1%):")
display(selected_weights[selected_weights > 0.001].sort_values(ascending=False))

# --- 2. Calculate Historical Performance ---
# Use the ACTUAL returns from the test period, not the predicted returns
test_dates_start = features_and_target_df.index[val_end + LOOKBACK_WINDOW]

# Removed redundant calculations. Assuming 'price_df_pd' is the correct DataFrame.
actual_returns_test = price_df_pd.loc[test_dates_start:, asset_tickers].pct_change().dropna()
benchmark_returns_test = price_df_pd.loc[test_dates_start:, BENCHMARK_TICKER].pct_change().dropna()

# Align indices before matrix multiplication
aligned_weights = selected_weights.values
aligned_returns, aligned_benchmark = actual_returns_test.align(benchmark_returns_test, join='inner', axis=0)

portfolio_historical_returns = (aligned_returns * aligned_weights).sum(axis=1)

# --- 3. Calculate Performance Metrics ---
def calculate_metrics(returns, risk_free_rate=0.02):
    """Calculates key performance metrics for a series of returns."""
    if returns.empty:
        return pd.Series({k: 'N/A' for k in ['Cumulative Return', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio', 'Max Drawdown']})

    cumulative_return = (1 + returns).prod() - 1
    annualized_return = returns.mean() * 252
    annualized_volatility = returns.std() * np.sqrt(252)
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else 0

    # Max Drawdown Calculation
    cumulative_wealth = (1 + returns).cumprod()
    peak = cumulative_wealth.cummax()
    drawdown = (cumulative_wealth - peak) / peak
    max_drawdown = drawdown.min()

    return pd.Series({
        'Cumulative Return': f"{cumulative_return:.2%}",
        'Annualized Return': f"{annualized_return:.2%}",
        'Annualized Volatility': f"{annualized_volatility:.2%}",
        'Sharpe Ratio': f"{sharpe_ratio:.2f}",
        'Max Drawdown': f"{max_drawdown:.2%}"
    })

# Define the risk-free rate as a constant for clarity
RISK_FREE_RATE = 0.02
portfolio_metrics = calculate_metrics(portfolio_historical_returns, risk_free_rate=RISK_FREE_RATE)
benchmark_metrics = calculate_metrics(aligned_benchmark, risk_free_rate=RISK_FREE_RATE)

performance_summary = pd.DataFrame({'Optimized Portfolio': portfolio_metrics, 'Benchmark (SPY)': benchmark_metrics})

print("\nBacktest Performance Summary:")
display(performance_summary)

# --- 5. Visualize Performance ---
plt.figure(figsize=(14, 8))

# Use the pre-aligned returns series for consistency and robustness
cumulative_portfolio_returns = (1 + portfolio_historical_returns).cumprod()
cumulative_benchmark_returns = (1 + aligned_benchmark).cumprod()

plt.plot(cumulative_portfolio_returns.index, cumulative_portfolio_returns, label='Optimized Portfolio', lw=2)
plt.plot(cumulative_benchmark_returns.index, cumulative_benchmark_returns, label='Benchmark (SPY)', linestyle='--', lw=2)

plt.title('Backtest: Optimized Portfolio vs. Benchmark (SPY)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Growth of $1', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# export the weights to a csv file
selected_weights.to_csv('highest_sharpe_portfolio_weights.csv')